In [1]:
import torch
from models import Orchestrator #Args  # Assurez-vous que Args est bien défini ou remplacez-le par vos paramètres
from config import Args
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import onnx 

# onnx 1.16.1
# to install onnx need to enable long path in file system regedit

2024-11-11 10:22:02.638 | INFO     | config:<module>:16 - PROJ_ROOT path is: /teamspace/studios/this_studio/mosquito-classification


In [2]:
# Étape 1 : Charger les arguments nécessaires (par exemple, à partir d’un fichier de config)
args = Args(wandb_entity="votre_nom_entité")  # Assurez-vous de définir tous les paramètres requis dans Args

In [3]:
input_data = './KO.png'

In [4]:
input_data=Image.open(input_data).convert('RGB') 

In [5]:
# Étape 2 : Charger le modèle
model = Orchestrator(args)
model.load_state_dict(torch.load("model_weights.pth"))

C:\Users\djibr\AppData\Local\Temp\ipykernel_11256\1437739450.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model_weights.pth"))


<All keys matched successfully>

In [6]:
model.eval()

Orchestrator(
  (f1score_train_macro): MulticlassF1Score()
  (f1score_valid_macro): MulticlassF1Score()
  (f1score_test_macro): MulticlassF1Score()
  (accuracy_train_macro): MulticlassAccuracy()
  (accuracy_valid_macro): MulticlassAccuracy()
  (accuracy_test_macro): MulticlassAccuracy()
  (aucpr_train_macro): MulticlassAveragePrecision()
  (aucpr_valid_macro): MulticlassAveragePrecision()
  (confmatrix_train): MulticlassStatScores()
  (confmatrix_valid): MulticlassStatScores()
  (confmatrix_test): MulticlassStatScores()
  (model): Classifier(
    (model): DenseNet(
      (features): Sequential(
        (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu0): ReLU(inplace=True)
        (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (denseblock1): _DenseBlock(
          (denselayer1): _DenseLayer(
     

In [7]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Redimensionner l'image à 224x224
    transforms.ToTensor(),           # Convertir l'image en tenseur
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalisation
])

In [8]:
input_data= transform(input_data) 

In [9]:
# Ajouter une dimension pour le batch
input_data = input_data.unsqueeze(0)  # Forme : [1, 3, 224, 224]

In [11]:
# Étape 3 : Préparer les données d'entrée pour la prédiction
#input_data = torch.randn(1, 3, 224, 224)  # Adapter la taille selon le modèle (par ex. 224x224 pour DenseNet)

In [10]:
# Étape 4 : Faire la prédiction
with torch.no_grad():
    output = model(input_data)
    probabilities = F.softmax(output, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1)
    #print(predicted_class.item())""
    if predicted_class.item() == 0:
        print("le moustique est un AE")
    elif predicted_class.item() == 1:
        print("le moustique est un AL")
    elif predicted_class.item() == 2:
        print("le moustique est un JA")
    elif predicted_class.item() == 3:
        print("le moustique est un KO")
    else:
         raise NotImplementedError()

le moustique est un JA


In [12]:
# torch_model = model
# torch_input = torch.randn(1,3,224,224)
# onnx_program = torch.onnx.dynamo_export(torch_model,torch_input)

In [11]:
dummy_input = torch.randn(1, 3, 224, 224)  # Example input
onnx_file_path = "model.onnx"

torch.onnx.export(
    model,                  # the model being run
    dummy_input,           # model input (or a tuple for multiple inputs)
    onnx_file_path,        # where to save the model
    export_params=True,    # store the trained parameter weights inside the model file
    opset_version=11,      # ONNX version to export the model to
    do_constant_folding=True,  # whether to execute constant folding for optimization
    input_names=['input'],  # the model's input names
    output_names=['output'],  # the model's output names
    dynamic_axes={          # variable-length axes
        'input': {0: 'batch_size'},  # batch size can vary
        'output': {0: 'batch_size'}
    }
)


In [14]:
import onnxruntime as ort

session = ort.InferenceSession('model.onnx')

In [15]:
# Get model input details
input_name = session.get_inputs()[0].name
input_shape = session.get_inputs()[0].shape
input_type = session.get_inputs()[0].type

print(f"Input name: {input_name}")
print(f"Input shape: {input_shape}")
print(f"Input type: {input_type}")


Input name: input
Input shape: ['batch_size', 3, 224, 224]
Input type: tensor(float)


In [16]:
onnx_output = session.run(None, {input_name: input_data})

RuntimeError: Input must be a list of dictionaries or a single numpy array for input 'input'.

In [ ]:
numpy_data = input_data.numpy()



In [21]:
outputs = session.run(None, {input_name: numpy_data})

In [22]:
outputs

[array([[ 0.2000307 , -0.4451368 ,  0.22326106, -0.8304324 ]],
       dtype=float32)]

In [23]:
output

tensor([[ 0.2000, -0.4451,  0.2233, -0.8304]])

In [25]:
new_probabilities = F.softmax(outputs, dim=1)
new_predicted_class = torch.argmax(new_probabilities, dim=1)

AttributeError: 'list' object has no attribute 'softmax'

In [28]:
from scipy.special import softmax

In [29]:
x = outputs

In [30]:
import numpy as np

In [31]:
n_softmax = np.exp(x) / np.sum(np.exp(x))
print(n_softmax)

[[[0.34424374 0.18058154 0.35233426 0.12284051]]]


In [32]:
probabilities

tensor([[0.3442, 0.1806, 0.3523, 0.1228]])

In [ ]:
# nummpy softmax practically same as tensorflow and scipy is practically the same too , there is a slight difference 0.00000001

In [33]:
sci_softmax = softmax(x)
print(sci_softmax)

[[[0.34424374 0.18058155 0.35233426 0.12284052]]]


In [34]:
x

[array([[ 0.2000307 , -0.4451368 ,  0.22326106, -0.8304324 ]],
       dtype=float32)]

In [35]:
output

tensor([[ 0.2000, -0.4451,  0.2233, -0.8304]])

In [36]:
input_data

tensor([[[[1.1700, 1.1700, 1.1700,  ..., 1.1700, 1.1700, 1.1700],
          [1.1700, 1.1700, 1.1700,  ..., 1.1700, 1.1700, 1.1700],
          [1.1529, 1.1529, 1.1529,  ..., 1.1015, 1.1015, 1.1187],
          ...,
          [1.1872, 1.1872, 1.2043,  ..., 1.1187, 1.1187, 1.1187],
          [1.1700, 1.1700, 1.1700,  ..., 1.1700, 1.1700, 1.1700],
          [1.1700, 1.1700, 1.1700,  ..., 1.1700, 1.1700, 1.1700]],

         [[1.3081, 1.3081, 1.3081,  ..., 1.3081, 1.3081, 1.3081],
          [1.3081, 1.3081, 1.3081,  ..., 1.3081, 1.3081, 1.3081],
          [1.2906, 1.2906, 1.2906,  ..., 1.2381, 1.2556, 1.2556],
          ...,
          [1.3431, 1.3431, 1.3431,  ..., 1.2731, 1.2556, 1.2556],
          [1.3081, 1.3081, 1.3081,  ..., 1.3081, 1.3081, 1.3081],
          [1.3081, 1.3081, 1.3081,  ..., 1.3081, 1.3081, 1.3081]],

         [[1.4200, 1.4200, 1.4200,  ..., 1.4200, 1.4200, 1.4200],
          [1.4200, 1.4200, 1.4200,  ..., 1.4200, 1.4200, 1.4200],
          [1.4025, 1.4025, 1.4025,  ..., 1